# Benchmark Summary
This notebook aggregates runtime measurements for multiple circuit families and highlights backend crossover points.

In [ ]:
import sys, pathlib
sys.path.append(str(pathlib.Path('..').resolve().parent))
from benchmarks.circuits import (
    ghz_circuit, qft_circuit, grover_circuit, random_circuit,
    w_state_circuit, graph_state_circuit,
)
from benchmarks.backends import (
    StatevectorAdapter, StimAdapter, MPSAdapter,
    DecisionDiagramAdapter, AerStatevectorAdapter,
    AerMPSAdapter, MQTDDAdapter,
)
from benchmarks.runner import BenchmarkRunner
from quasar import SimulationEngine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

families = {
    'ghz': ghz_circuit,
    'qft': qft_circuit,
    'grover': grover_circuit,
    'random': lambda n: random_circuit(n, seed=42),
    'w': w_state_circuit,
    'graph': lambda n: graph_state_circuit(n, degree=3, seed=42),
}
qubit_counts = {
    'ghz': range(2, 13, 2),
    'qft': range(2, 13, 2),
    'grover': range(2, 13, 2),
    'random': range(2, 13, 2),
    'w': range(2, 13, 2),
    'graph': range(4, 13, 2),
}

backends = [StatevectorAdapter(), StimAdapter(), MPSAdapter(), DecisionDiagramAdapter()]
for cls in [AerStatevectorAdapter, AerMPSAdapter, MQTDDAdapter]:
    try:
        backends.append(cls())
    except Exception:
        pass

runner = BenchmarkRunner()
engine = SimulationEngine()

for name, fn in families.items():
    for n in qubit_counts[name]:
        circ = fn(n)
        plan = engine.planner.plan(circ)
        backend = plan.steps[0].backend.name if plan.steps else None
        print(f"{name} n={n} -> {backend}")
        for b in backends:
            try:
                rec = runner.run(circ, b)
                rec['qubits'] = n
                rec['family'] = name
                rec['selected_backend'] = None
            except Exception:
                continue
        try:
            rec = runner.run_quasar(circ, engine)
            rec['qubits'] = n
            rec['family'] = name
            rec['selected_backend'] = backend
        except Exception:
            runner.results.append({'framework':'quasar','time':float('nan'),'result':None,'qubits':n,'family':name,'selected_backend':backend})

df = pd.DataFrame(runner.results)
print(df[df['framework']=='quasar'][['family','qubits','selected_backend']])

sns.relplot(data=df, x='qubits', y='time', hue='framework', col='family', kind='line', facet_kws={'sharey': False})
plt.yscale('log')
plt.show()

# Identify crossover points where the fastest backend changes
def find_crossover(group):
    g = group.sort_values('qubits')
    best = g.loc[g.groupby('qubits')['time'].idxmin()][['qubits','framework']]
    change = best[best['framework'] != best['framework'].shift()].copy()
    return change
crossover = df.groupby('family').apply(find_crossover)
crossover
